# All required pip installs

In [ ]:
!pip install langdetect 
!pip3 install googletrans==3.1.0a0
!pip install autocorrect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Libraries

In [ ]:
import numpy as np
import pandas as pd
import re
import string
from googletrans import Translator
# from google.transliteration import transliterate_text
from langdetect import detect_langs

ModuleNotFoundError: ignored

# Basic Pre - Processing

In [ ]:
val = -1

In [ ]:
df_real = pd.read_csv("/content/kannada_sentiment.csv")
df_real.head()

,Sentiment,Cross_Lingual_Text
0,Negative,Sir nivu news helida hage heltiri sir
1,unknown state,Idu riyel rar
2,Negative,ಕಥೆ ಅರ್ದ ಅಗಿದೆ.ಅಶ್ವತ್ತಾಮ ತನ್ಣಗಾಯಗೊಂಡ ದೇಹವನ್ನು ...
3,Mixed feelings,ಕಥೆ ಸರಿಯಾಗಿ ತಿಳಿದುಕೊಳ್ಳಿ..... please don't wor...
4,Positive,Ashwathama. Vadhukkidhe.evathu. video. Nodithu...


# Data Cleaning

In [ ]:
df = df_real[:val].copy()

In [ ]:
def preprocessing(line) :
    # Remove words containing @
    line = re.sub(r'()@\w+', r'\1', line)

    line = line.translate(str.maketrans('', '', string.punctuation))
    
    # Remove remaining extra spaces
    line = re.sub(" +", " ", line).strip()

    emoj = re.compile("["
    u"\U0001F600-\U0001F64F"  # emoticons
    u"\U0001F300-\U0001F5FF"  # symbols & pictographs
    u"\U0001F680-\U0001F6FF"  # transport & map symbols
    u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
    u"\U00002500-\U00002BEF"  # chinese char
    u"\U00002702-\U000027B0"
    u"\U00002702-\U000027B0"
    u"\U000024C2-\U0001F251"
    u"\U0001f926-\U0001f937"
    u"\U00010000-\U0010ffff"
    u"\u2640-\u2642" 
    u"\u2600-\u2B55"
    u"\u200d"
    u"\u23cf"
    u"\u23e9"
    u"\u231a"
    u"\ufe0f"  # dingbats
    u"\u3030"
                  "]+", re.UNICODE)
    preprocessed_sentence = re.sub(emoj, '', line)

    return preprocessed_sentence

In [ ]:
df["Processed_text"] = df["Cross_Lingual_Text"][:val].apply(lambda Cross_Lingual_Text: preprocessing(Cross_Lingual_Text))
df.head()

,Sentiment,Cross_Lingual_Text,Processed_text
0,Negative,Sir nivu news helida hage heltiri sir,Sir nivu news helida hage heltiri sir
1,unknown state,Idu riyel rar,Idu riyel rar
2,Negative,ಕಥೆ ಅರ್ದ ಅಗಿದೆ.ಅಶ್ವತ್ತಾಮ ತನ್ಣಗಾಯಗೊಂಡ ದೇಹವನ್ನು ...,ಕಥೆ ಅರ್ದ ಅಗಿದೆಅಶ್ವತ್ತಾಮ ತನ್ಣಗಾಯಗೊಂಡ ದೇಹವನ್ನು ಗ...
3,Mixed feelings,ಕಥೆ ಸರಿಯಾಗಿ ತಿಳಿದುಕೊಳ್ಳಿ..... please don't wor...,ಕಥೆ ಸರಿಯಾಗಿ ತಿಳಿದುಕೊಳ್ಳಿ please dont worng gui...
4,Positive,Ashwathama. Vadhukkidhe.evathu. video. Nodithu...,Ashwathama Vadhukkidheevathu video Nodithunanu...


In [ ]:
def Kannada_Translator(line):
    translator = Translator()
    line = translator.translate(line,dest="kn")
    return line.text

In [ ]:
df["Translated_text"] = df["Processed_text"][:val].apply(lambda Processed_text: Kannada_Translator(Processed_text))
df.head()

KeyboardInterrupt: ignored

In [ ]:
df.dropna(subset=["Translated_text"], inplace = True)

In [ ]:
df.replace("Negative",0,inplace=True)
df.replace("Positive",1,inplace=True)
df.replace("Mixed feelings",2,inplace=True)
df.replace("unknown state",3,inplace=True)
df.replace("not-Kannada",4,inplace=True)
df.head()

In [ ]:
def convert_to_kannada(preprocessed_sentence, translator):
    try :
        kannada_words = []
        english_words = []

        words = preprocessed_sentence.split(" ")
        words = [word.strip("?") for word in words]

        for word in words:
            if word.isnumeric() :
                continue
            lan=[{lang.lang: lang.prob} for lang in detect_langs(word)]

            if 'kn' in lan[0]:
                kannada_words.append(word)
            else :
                spell = Speller(lang='en')
                word = spell(word)
                english_words.append(word)

        translated_sentence = []
        if len(english_words)==0:
            return preprocessed_sentence

        for word in words :
            if word in english_words :
                new_word = translator.translate(word, dest = "kn").text
                translated_sentence.append(new_word)
            else :
                translated_sentence.append(word)
                
        translated_sentence = " ".join(translated_sentence)
        translation = translator.translate(translated_sentence, dest='en')
        translation = translator.translate(translation.text, dest='kn')
        return translation.text
    except:
        print("Error word", word)
        print("error sentence", preprocessed_sentence)
        return np.nan


In [ ]:
from autocorrect import Speller
translator = Translator()

In [ ]:
df.dropna(inplace = True, axis="columns")

In [ ]:
new_df = df["Translated_text"].copy()

new_df["FULL_kannada"] = df["Translated_text"].apply(lambda Translated_text: convert_to_kannada(Translated_text, translator))

In [ ]:
df["spell_correct"] = new_df["FULL_kannada"]

# Random generation

In [ ]:
def convert_to_kannada2(preprocessed_sentence, translator):
    kannada_words = []
    english_words = []

    words = preprocessed_sentence.split(" ")
    words = [word.strip("?") for word in words]

    for word in words:
        if word.isnumeric() :
            continue
        lan=[{lang.lang: lang.prob} for lang in detect_langs(word)]

        if 'kn' in lan[0]:
            kannada_words.append(word)
        else :
            spell = Speller(lang='en')
            word = spell(word)
            english_words.append(word)

    translated_sentence = []
    if len(english_words)==0:
        return preprocessed_sentence

    for word in words :
        if word in english_words :
            new_word = translator.translate(word, dest = "kn").text
            translated_sentence.append(new_word)
        else :
            translated_sentence.append(word)
            
    
    return " ".join(translated_sentence)


In [ ]:
def jumbled(text):
    translation = translator.translate(text, dest='en')
    translation = translator.translate(translation.text, dest='kn')
    return translation.text

In [ ]:
sentence = 'Can you please stop showing useles informatoin 😑😑😑'
level1 = preprocessing(sentence)
level2 = Kannada_Translator(level1)
level3 = convert_to_kannada2(level2, translator)
level4 = jumbled(level3)

In [ ]:
print(level1, "\n",level2, "\n",level3, "\n",level4)

In [ ]:
df = pd.read_csv("/content/kannada_sentiment_processed_full_kannada (1).csv")

In [ ]:
df.head()

,Unnamed: 0,Sentiment,Cross_Lingual_Text,Processed_text,Translated_text,FULL_kannada
0,0,0,Sir nivu news helida hage heltiri sir,Sir nivu news helida hage heltiri sir,ಸರ್ ನೀವು ನ್ಯೂಸ್ ಹೇಳಿದ ಹಾಗೆ ಹೇಳ್ತಿರಿ ಸರ್,ಸರ್ ನೀವು ನ್ಯೂಸ್ ಹೇಳಿದ ಹಾಗೆ ಹೇಳ್ತಿರಿ ಸರ್
1,1,3,Idu riyel rar,Idu riyel rar,ಇದು ರಿಯೆಲ್ ರಾರ್,ಇದು ರಿಯೆಲ್ ರಾರ್
2,2,0,ಕಥೆ ಅರ್ದ ಅಗಿದೆ.ಅಶ್ವತ್ತಾಮ ತನ್ಣಗಾಯಗೊಂಡ ದೇಹವನ್ನು ...,ಕಥೆ ಅರ್ದ ಅಗಿದೆಅಶ್ವತ್ತಾಮ ತನ್ಣಗಾಯಗೊಂಡ ದೇಹವನ್ನು ಗ...,ಕಥೆ ಅರ್ದ ಅಗಿದೆಅಶ್ವತ್ತಾಮ ತನ್ಣಗಾಯಗೊಂಡ ದೇಹವನ್ನು ಗ...,ಕಥೆ ಅರ್ದ ಅಗಿದೆಅಶ್ವತ್ತಾಮ ತನ್ಣಗಾಯಗೊಂಡ ದೇಹವನ್ನು ಗ...
3,3,1,Ashwathama. Vadhukkidhe.evathu. video. Nodithu...,Ashwathama Vadhukkidheevathu video Nodithunanu...,ಅಶ್ವಥಮ ವಧುಕ್ಕಿದೀವತು ವಿಡಿಯೋ ನೋಡಿತುನನು ಜೈ ಬಾಬಾಜಿ,ಅಶ್ವಥಮ ವಧುಕ್ಕಿದೀವತು ವಿಡಿಯೋ ನೋಡಿತುನನು ಜೈ ಬಾಬಾಜಿ
4,4,0,ಹೌದು ಹೌದು ನಿನ್ನೆ ನಮ್ಮನಿಗೂ ಬಂದಿದ........ ಚಾ ಕುಡ...,ಹೌದು ಹೌದು ನಿನ್ನೆ ನಮ್ಮನಿಗೂ ಬಂದಿದ ಚಾ ಕುಡ್ಕಾ ಹೊದ,ಹೌದು ಹೌದು ನಿನ್ನೆ ನಮ್ಮನಿಗೂ ಬಂದಿದ ಚಾ ಕುಡ್ಕಾ ಹೊದ,ಹೌದು ಹೌದು ನಿನ್ನೆ ನಮ್ಮನಿಗೂ ಬಂದಿದ ಚಾ ಕುಡ್ಕಾ ಹೊದ


In [ ]:
print(df['Sentiment'].value_counts()[0])
print(df['Sentiment'].value_counts()[1])
print(df['Sentiment'].value_counts()[2])
print(df['Sentiment'].value_counts()[3])
print(df['Sentiment'].value_counts()[4])


1082
2900
547
718
1061
